In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/dataset/avocado.csv')

# Drop the 'Unnamed' and 'Date' columns
df.drop(columns=['Unnamed: 0', 'Date'], inplace=True)

# Handle Missing Values
# Numerical columns: impute with mean
# Categorical columns: impute with most frequent
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Impute missing values
for col in numerical_cols:
    df[col].fillna(df[col].mean(), inplace=True)

# Convert 'type' column to category if it's not already and apply Label Encoding
if 'type' in categorical_cols:
    df['type'] = pd.Categorical(df['type'])
    df['type'] = df['type'].cat.codes

# One-Hot Encoding for the 'region' column
df = pd.get_dummies(df, columns=['region'], drop_first=True)

# After one-hot encoding, all columns should be numeric.
# If any non-numeric columns remain, this would identify them.
non_numeric_columns = df.select_dtypes(include=['object']).columns
if non_numeric_columns.any():
    raise Exception(f"Non-numeric columns are present: {non_numeric_columns}")

# Handle outliers by replacing them with the mean of the column
for col in numerical_cols:
    if col != 'type':  # 'type' is now encoded, no longer numerical
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

# Separate features and target variable
y = df['AveragePrice']
X = df.drop('AveragePrice', axis=1)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Combine the scaled features with the target variable for train and test sets
train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
train_df['AveragePrice'] = y_train.reset_index(drop=True)

test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
test_df['AveragePrice'] = y_test.reset_index(drop=True)

# Save the preprocessed train and test sets into CSV files
train_df.to_csv('//content/drive/MyDrive/dataset/train_avocado_1.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/dataset/test_avocado_1.csv', index=False)
